# Plant Species Care Guides — Data Pull Notebook

This notebook fetches care-guide JSON from Perenual’s API and saves one file per species ID.
It’s simple and polite to the API: single API key, light rate-limiting, retries on 429/5xx, and a safe stop when daily quota is likely hit. It’s also resume-friendly—existing files are skipped.  

Before each time running, teammates can fetch from our github reporsitory first, then go through step 1 - 4, which means simply click "Run All" is enough.

## Step 1 - Configuration (range, output paths)



Set the API key, the ID range to fetch, basic rate-limit settings, and where to save the files.

In [6]:
# Put a single API key here
API_KEY = "sk-3ksU68b046f66e1f912089"

# Fetch range (start small when testing, then scale up)
START_ID = 1
END_ID   = 3000

# Rate limiting & retry behavior
SLEEP_BETWEEN = 1        # seconds to sleep after each request
MAX_RETRIES   = 5
BACKOFF_BASE  = 1.6

# Output directory & filename pattern (pathlib adapts across OSes)
from pathlib import Path
OUT_DIR = Path("01_raw_data/02_care_guide")
FILENAME_PATTERN = "plant_species_care_guide_{species_id}.json"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Key: ****{API_KEY[-6:]}, Range: {START_ID}~{END_ID}, Output dir: {OUT_DIR.resolve()}")

Key: ****912089, Range: 1~3000, Output dir: E:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\01_raw_data\02_care_guide


## Step 2 - Some Small Helper Functions



Minimal helpers to save JSON and build output file paths.

In [7]:
import time
import json
from typing import Optional, Dict, Any
import requests

def save_json(path: Path, data: Dict[str, Any]):
    # Ensure the parent directory exists, then save UTF-8 JSON
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

def build_filepath(species_id: int) -> Path:
    # One JSON file per species_id
    return OUT_DIR / FILENAME_PATTERN.format(species_id=species_id)

## Step 3 - Fetch one species (with retries & safe-stop)



Calls the care-guide endpoint, handles 404s (placeholder), retries on temporary errors, and stops safely when rate limits/quota are inferred.

In [8]:
def fetch_species_care_guide(species_id: int, api_key: str) -> Optional[dict]:
    """
    Success -> returns a JSON dict
    404     -> returns {"__missing__": True, "id": species_id} (placeholder)
    None    -> signals the main loop to stop (e.g., quota/IP limit, repeated 429, or x-ratelimit-remaining == 0)
    """
    # API docs show the care-guide endpoint over HTTP
    url = "http://perenual.com/api/species-care-guide-list"
    attempt = 0
    consecutive_429 = 0

    while attempt <= MAX_RETRIES:
        try:
            resp = requests.get(
                url,
                params={"species_id": species_id, "key": api_key},
                headers={"accept": "application/json"},
                timeout=30
            )

            # Check remaining-quota hint if the server provides it
            remaining = resp.headers.get("x-ratelimit-remaining")
            if remaining is not None:
                try:
                    if int(remaining) <= 0:
                        print(f"[ID {species_id}] x-ratelimit-remaining=0 → stopping safely (quota likely reached).")
                        return None
                except ValueError:
                    pass  # ignore if not an integer

            if resp.status_code == 200:
                return resp.json()

            if resp.status_code == 404:
                print(f"[ID {species_id}] 404: this species may not have a care guide; saving a placeholder.")
                return {"__missing__": True, "id": species_id}

            if resp.status_code == 429:
                consecutive_429 += 1
                if consecutive_429 >= 3:
                    print(f"[ID {species_id}] 429 occurred {consecutive_429} times in a row → stopping safely (rate/IP limit).")
                    return None
                wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
                print(f"[ID {species_id}] 429 Too Many Requests — waiting {wait:.1f}s before retrying…")
                time.sleep(wait)
                attempt += 1
                continue

            if resp.status_code in (500, 502, 503, 504):
                wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
                print(f"[ID {species_id}] {resp.status_code} server error — waiting {wait:.1f}s then retrying…")
                time.sleep(wait)
                attempt += 1
                continue

            # For other unexpected status codes, log a short snippet for context
            print(f"[ID {species_id}] HTTP {resp.status_code}: {resp.text[:200]}")
            return {"__error_status__": resp.status_code, "id": species_id}

        except requests.RequestException as e:
            wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
            print(f"[ID {species_id}] Network error ({type(e).__name__}): {e}. Waiting {wait:.1f}s then retrying…")
            time.sleep(wait)
            attempt += 1

    print(f"[ID {species_id}] Retries exhausted. Skipping this one.")
    return {"__error_retries_exhausted__": True, "id": species_id}

## Step 4 - Main loop (resume-friendly, skips existing files)



Loops through the ID range, skips files that already exist, lightly sleeps between requests, and stops when the fetcher signals None.

In [9]:
from tqdm.auto import tqdm
import random

downloaded = 0
skipped = 0

for species_id in tqdm(range(START_ID, END_ID + 1), desc="Downloading Care Guides"):
    fp = build_filepath(species_id)
    if fp.exists():
        skipped += 1
        continue

    data = fetch_species_care_guide(species_id, API_KEY)
    if data is None:
        print("Quota/rate-limit inferred → stopping safely.")
        break

    save_json(fp, data)
    downloaded += 1
    # Small random jitter helps avoid synchronized spikes
    time.sleep(SLEEP_BETWEEN + random.uniform(0.0, 0.6))

print(f"New downloads: {downloaded}, skipped (already exists): {skipped}. Output dir: {OUT_DIR.resolve()}")

[ID 298] 429 Too Many Requests — waiting 1.0s before retrying…
[ID 298] 429 Too Many Requests — waiting 1.8s before retrying…


[ID 298] 429 occurred 3 times in a row → stopping safely (rate/IP limit).
Quota/rate-limit inferred → stopping safely.
New downloads: 0, skipped (already exists): 297. Output dir: E:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\01_raw_data\02_care_guide


## Step 5 (Optional) - Pack and download as a zip (for Colab)



When running on Colab, zip the folder and download it locally. On a local machine, just use your file manager.

In [10]:
# # Only needed on Google Colab; skip on local environments
# import shutil
# from google.colab import files
#
# shutil.make_archive("02_species_care_guide", 'zip', "01_raw_data/02_care_guide")
# files.download("02_species_care_guide.zip")